In [403]:
from pytube import YouTube
from pytube import Playlist
from pytube import Channel
import os
import re

In [325]:
def input_url(URL = 'https://www.youtube.com/watch?v=DYzT-Pk6Ogw'):
    yt = YouTube(URL)
    title = yt.title
    print(f'!Download : {title}')
    return yt, title

In [326]:
yt, title = input_url()

!Download : It do go down


> Average bitrate (ABR) refers to the average amount of data transferred per unit of time, usually measured per second, commonly for digital music or video. An MP3 file, for example, that has an average bit rate of 128 kbit/s transfers, on average, 128,000 bits every second.

> which is progressive (video with audio)

In [320]:
def get_itag(yt=yt) :
    tag_audio = list(yt.streams.filter(only_audio=True))
    tag_video = list(yt.streams.filter(file_extension='mp4'))

    audio_itag = {}
    video_itag = {}
    video_itag_nosound = {}
    all_res = []

    i = 1
    for stream in tag_audio :
        stream = f'{stream}'
        stream = stream.split(' ')
        itag, abr = stream[1], stream[3]
        itag_num = itag.split('"')[1]
        abr_num = abr.split('"')[1]
        
        audio_itag[i] = {abr_num:itag_num}
        i+=1

    j = 1
    for stream in tag_video :
        stream = f'{stream}'
        stream = stream.split(' ')
        itag, res, fps, pro = stream[1], stream[3], stream[4], stream[-2]

        if 'True' in pro:
            if 'res' in res :
                itag_num = itag.split('"')[1]
                res_num = res.split('"')[1]
                fps_num = fps.split('"')[1]
                
                video_itag[j] = {f'{res_num}-{fps_num}':itag_num}
                j+=1
        
        else :
            if 'res' in res :
                if res not in all_res :
                    all_res.append(res)

                    itag_num = itag.split('"')[1]
                    res_num = res.split('"')[1]
                    fps_num = fps.split('"')[1]

                    video_itag_nosound[j] = {f'no-sound-{res_num}-{fps_num}':itag_num}
                    j+=1

    return audio_itag , video_itag, video_itag_nosound

In [321]:
audio_itag , video_itag, video_itag_nosound = get_itag(yt)

In [315]:
def show_choice_audio_only(audio_itag=audio_itag):
    target = 'audio only'
    print(f'> choice of {target}')
    for index in range(len(audio_itag)):
        for abr, itag in audio_itag[index+1].items() :
            print(f' {index+1}. abr = {abr}')

In [322]:
def show_choice_video(video_itag=video_itag, video_itag_nosound=video_itag_nosound):
    target = 'videos'
    print(f'> choice of {target}')

    for index in range(len(video_itag)):
        for res_fps, itag in video_itag[index+1].items() :
            print(f' {index+1}. res = {res_fps}')
        
    for index in range(len(video_itag_nosound)):
        for res_fps, itag in video_itag_nosound[index+len(video_itag)+1].items() :
            print(f' {index+1+len(video_itag)}. res = {res_fps}')

In [327]:
print(title)
show_choice_audio_only(audio_itag), 
show_choice_video(video_itag,video_itag_nosound)

It do go down
> choice of audio only
 1. abr = 48kbps
 2. abr = 128kbps
 3. abr = 50kbps
 4. abr = 70kbps
 5. abr = 160kbps
> choice of videos
 1. res = 360p-30fps
 2. res = no-sound-480p-30fps
 3. res = no-sound-360p-30fps
 4. res = no-sound-240p-30fps
 5. res = no-sound-144p-30fps


In [336]:
video_itag_nosound[2].items()

dict_items([('no-sound-480p-30fps', '135')])

In [440]:
def create_dir(specific=''):
    folder_name = 'Youtube_download'
    desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop') 
    PATH = f'{desktop}\\{folder_name}\\{specific}'
    os.makedirs(PATH, exist_ok = True)
    return PATH

In [370]:
PATH = create_dir()

In [441]:
itag = int(list(audio_itag[1].values())[0])
itag

139

In [447]:
def download_by_itag(PATH,itag=itag,type='mp4',yt=None,title=None):
    stream = yt.streams.get_by_itag(itag)
    stream.download(PATH,f'{title}.{type}')


path can't using \/:*?"<>|

In [448]:
def clean_path(title):
    save_title = str(title)
    to_replace = {'/':'',':':'','*':'','?':'','"':'','<':'','>':'','|':''}
    save_title = re.sub(r'[\/:*?"<>|]', lambda x:to_replace[x.group(0)],save_title)
    return save_title

In [470]:
def single_video_download(URL=None):
    '''
    download a single video from youtube URL
    '''
    
    yt, title = input_url(URL)

    audio_itag , video_itag, video_itag_nosound = get_itag(yt)
    
    type = input('what do you want ?\n 1. audio only\n 2. video\n :')

    save_title = clean_path(title) 
    
    if type == '1' :
        print('\nwhich quality you want? (select only one)')
        show_choice_audio_only(audio_itag)
        qt = int(input(' :'))
        
        itag = int(list(audio_itag[qt].values())[0])
        
        PATH = create_dir()
        print(f'\ndownloading.. {title}')


        download_by_itag(PATH,itag,'mp3',yt,save_title)
        print(f'download {title} successfully!! check in folder Youtube_download on your Desktop')
    
    else :
        print('\nwhich quality you want?')
        show_choice_video(video_itag,video_itag_nosound)
        qt = int(input(' :'))
        
        if qt <= len(video_itag):
            itag = int(list(video_itag[qt].values())[0])
        else:
            itag = int(list(video_itag_nosound[qt].values())[0])
        
        PATH = create_dir()
        print(f'\ndownloading.. {title}')
        download_by_itag(PATH,itag,'mp4',yt,save_title)
        print(f'download {title} successfully!! check in folder Youtube_download on your Desktop')
        

In [471]:
def multi_video_download(URL=None,num_videos=0,recenly=True):
    '''
    using URL from the playlist or Channel to download videos
    : if using URL from playlist recenly=True mean from the top to bottom
    : if using URl from Channel recenly=True mean recenly videos
    '''
    URLs = []
    
    try :
        p = Playlist(URL)
        print(f'!Download from {p.title}')
        folder_name = p.title

        if recenly :
            for URL in p.video_urls[:num_videos]:
                URLs.append(URL) 
        else :
            for URL in p.video_urls:
                URLs.append(URL)
            URLs = URLs[::-1]
            URLs = URLs[:num_videos]
        
    except :
        c = Channel(URL)
        print(f'!Download from {c.channel_name}')
        folder_name = c.channel_name
        
        if recenly :
            for URL in c.video_urls[:num_videos]:
                URLs.append(URL) 
        else :
            for URL in c.video_urls:
                URLs.append(URL)
            URLs = URLs[::-1]
            URLs = URLs[:num_videos]

    type = input('what do you want ?\n 1. audio only\n 2. video\n :')
    folder_name = clean_path(folder_name)
    PATH = create_dir(folder_name)

    if type == '1':
        for URL in URLs :
            yt, title =input_url(URL)
            title = clean_path(title)
            stream = yt.streams.get_audio_only()
            stream.download(PATH,f'{title}.mp3')
            print(f'download {title} successful!! check in folder Youtube_download on your Desktop\n')
            
    else :
        qt = input('low or high resolution ?\n 1. low\n 2. high\n :')
        if qt == '1':
            for URL in URLs :
                yt, title =input_url(URL)
                title = clean_path(title)
                stream = yt.streams.get_lowest_resolution()
                stream.download(PATH,f'{title}.mp4')
                print(f'download {title} successfully!! check in folder Youtube_download on your Desktop\n')
        else :
            for URL in URLs :
                yt, title =input_url(URL)
                title = clean_path(title)
                stream = yt.streams.get_highest_resolution()
                stream.download(PATH,f'{title}.mp4')
                print(f'download {title} successfully!! check in folder Youtube_download on your Desktop\n')
    
    print(f'!!Download all files from {folder_name} sucessfully')
    

### USE APP HERE !!

In [472]:
single_video_download('https://www.youtube.com/watch?v=0hBsNxWfC74')

!Download : Initial D nascar

which quality you want? (select only one)
> choice of audio only
 1. abr = 48kbps
 2. abr = 128kbps
 3. abr = 50kbps
 4. abr = 70kbps
 5. abr = 160kbps

downloading.. Initial D nascar
download Initial D nascar successfully!! check in folder Youtube_download on your Desktop


In [469]:
multi_video_download('https://www.youtube.com/c/EdSheeran/videos',num_videos=3,recenly=True)

!Download from Ed Sheeran
!Download : Ed Sheeran - 2steppin in a Cork dressing room
download Ed Sheeran - 2steppin in a Cork dressing room successful!! check in folder Youtube_download on your Desktop

!Download : Ed Sheeran - 2step (feat. Lil Baby) - [Official Behind The Scenes Video]
download Ed Sheeran - 2step (feat. Lil Baby) - [Official Behind The Scenes Video] successful!! check in folder Youtube_download on your Desktop

!Download : Ed Sheeran - 2step (feat. Lil Baby) - [Official Video]
download Ed Sheeran - 2step (feat. Lil Baby) - [Official Video] successful!! check in folder Youtube_download on your Desktop

